### Importing Libraries and setting the seed

In [ ]:
import numpy as np
import pandas as pd
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
%pip install xgboost
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, BatchNormalization, Activation,SimpleRNN, LSTM, GRU


np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
os.environ['PYTHONHASHSEED'] = str(42)


### **PD Speech Dataset: Signal Processing for Diagnosis**
### **Data Set Information**:
The data used in this study were gathered from 188 patients with PD (107 men and 81 women) with ages ranging from 33 to 87 (65.1Â±10.9) at the Department of Neurology in CerrahpaÅŸa Faculty of Medicine, Istanbul University. The control group consists of 64 healthy individuals (23 men and 41 women) with ages varying between 41 and 82 (61.1Â±8.9). During the data collection process, the microphone is set to 44.1 KHz and following the physicianâ€™s examination, the sustained phonation of the vowel /a/ was collected from each subject with three repetitions.
### **Attribute Information**:
Various speech signal processing algorithms including Time Frequency Features, Mel Frequency Cepstral Coefficients (MFCCs), Wavelet Transform based Features, Vocal Fold Features and TWQT features have been applied to the speech recordings of Parkinson's Disease (PD) patients to extract clinically useful information for PD assessment.

### Reading file and doing train, validate and test splitting

In [ ]:
file_path = 'pd_speech_features.csv'
data = pd.read_csv(file_path)
data.head()

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [ ]:
data.shape

(756, 755)

In [ ]:
data = data.iloc[1:]

# Ensuring that all data is numeric
data = data.apply(pd.to_numeric, errors='coerce')

# Handling missing values if any
data = data.dropna()

# Preprocessing the dataset
X = data.iloc[:, 1:-1].values  # Excluding 'id' and 'class' columns
y = data.iloc[:, -1].values   # 'class' column

# Normalizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshaping X for 1D CNN input
X_reshaped = np.expand_dims(X_scaled, axis=2)

# Splitting the dataset
X_train, X_temp, y_train, y_temp = train_test_split(X_reshaped, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

epochs = 25  # Number of epochs;
batch_size = 80  # Batch size;


### **Definition of Convolutional Neural Networks (CNNs)**
Convolutional Neural Networks (CNNs) are a class of deep neural networks, most commonly applied to analyzing visual imagery. They are particularly known for their ability to detect patterns and features in images through a process known as convolution, which involves sliding a filter or kernel over the input data to produce a feature map. This process captures local dependencies in the data, such as edges and textures in images. CNNs typically consist of a series of layers: convolutional layers for feature extraction, pooling layers for dimensionality reduction, and fully connected layers for classification or regression tasks.



### **CNNs in the Context of PD Speech Data**
When applied to datasets like PD (Parkinson's Disease) speech, CNNs can be adapted to process one-dimensional time-series data instead of two-dimensional image data. In this context, CNNs work by:

* Feature Extraction: Extracting relevant features from the speech signals, which may include aspects like frequency changes, amplitude variations, and temporal dynamics. These features are crucial in identifying speech characteristics that may be indicative of Parkinson's Disease.

* Capturing Temporal Patterns: Using 1D convolutional layers, CNNs can capture temporal patterns within the speech data. The convolution operation allows the network to recognize specific patterns in the speech signal that are consistently associated with PD symptoms.

* Handling Varied Input Lengths: Speech data often comes in varied lengths. CNNs can manage this through techniques like padding or cutting to ensure consistent input sizes or by using global pooling layers to handle inputs of varying dimensions.

* Classification: After feature extraction and pattern recognition, CNNs use one or more fully connected layers to classify the speech samples into categories (such as PD or non-PD) based on the learned features.

In summary, CNNs, when used for PD speech datasets, focus on automatically learning speech signal features that are relevant for distinguishing between healthy individuals and those affected by Parkinson's Disease. This involves adapting the network to handle one-dimensional, time-series data, focusing on temporal feature extraction, and employing classification layers to make predictions based on these features.

### Creating different types of CNNs and evaluating their results

In [ ]:
def create_cnn_model_1(input_shape):
    # Simple CNN model
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_model_2(input_shape):
    # CNN model with more layers
    model = Sequential([
        Conv1D(filters=64, kernel_size=5, input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_model_3(input_shape):
    # CNN model with different kernel sizes
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, input_shape=input_shape),
        Activation('relu'),
        Conv1D(filters=32, kernel_size=3),
        Activation('relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to train and evaluate a model
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    return test_loss, test_accuracy

# Creating models
input_shape = (X_train.shape[1], 1)
model_1 = create_cnn_model_1(input_shape)
model_2 = create_cnn_model_2(input_shape)
model_3 = create_cnn_model_3(input_shape)

# Training and evaluating models
loss_1, acc_1 = train_and_evaluate_model(model_1, X_train, y_train, X_test, y_test)
loss_2, acc_2 = train_and_evaluate_model(model_2, X_train, y_train, X_test, y_test)
loss_3, acc_3 = train_and_evaluate_model(model_3, X_train, y_train, X_test, y_test)

Epoch 1/10
12/12 [==============================] - 2s 65ms/step - loss: 0.8218 - accuracy: 0.6657 - val_loss: 0.3485 - val_accuracy: 0.8901
Epoch 2/10
12/12 [==============================] - 0s 36ms/step - loss: 0.4685 - accuracy: 0.7845 - val_loss: 0.3585 - val_accuracy: 0.8462
Epoch 3/10
12/12 [==============================] - 0s 35ms/step - loss: 0.3280 - accuracy: 0.8536 - val_loss: 0.3699 - val_accuracy: 0.8571
Epoch 4/10
12/12 [==============================] - 1s 48ms/step - loss: 0.3048 - accuracy: 0.8646 - val_loss: 0.3436 - val_accuracy: 0.8571
Epoch 5/10
12/12 [==============================] - 1s 45ms/step - loss: 0.2672 - accuracy: 0.8867 - val_loss: 0.3297 - val_accuracy: 0.8681
Epoch 6/10
12/12 [==============================] - 1s 44ms/step - loss: 0.2249 - accuracy: 0.9171 - val_loss: 0.3183 - val_accuracy: 0.8571
Epoch 7/10
12/12 [==============================] - 1s 48ms/step - loss: 0.1946 - accuracy: 0.9392 - val_loss: 0.3144 - val_accuracy: 0.8681
Epoch 8/10
12

In [ ]:
# Printing results
print("Model 1 - Loss:", loss_1, "Accuracy:", acc_1)
print("Model 2 - Loss:", loss_2, "Accuracy:", acc_2)
print("Model 3 - Loss:", loss_3, "Accuracy:", acc_3)

Model 1 - Loss: 0.3705005645751953 Accuracy: 0.8410596251487732
Model 2 - Loss: 0.41870689392089844 Accuracy: 0.8278145790100098
Model 3 - Loss: 0.45913276076316833 Accuracy: 0.8344370722770691


### **Definition of Artificial Neural Networks (ANNs)**
Artificial Neural Networks (ANNs) are a foundational construct in the field of machine learning and deep learning, inspired by the biological neural networks that constitute animal brains. An ANN is composed of interconnected units or nodes called artificial neurons, which loosely model the neurons in a biological brain. Each connection between neurons can transmit a signal from one neuron to another, and the receiving neuron processes the signal and signals downstream neurons connected to it. Neurons are organized in layers: input layers to receive signals, hidden layers to process signals, and an output layer to make a prediction or decision. ANNs are capable of learning complex patterns and relationships in data by adjusting the weights of connections through a process known as training.

### **ANNs in the Context of PD Speech Data**
In the context of PD (Parkinson's Disease) speech data analysis, ANNs are used to identify patterns and characteristics in speech that are indicative of Parkinson's Disease. Here’s how ANNs typically function in this scenario:

* Handling Sequential Data: Although ANNs are not inherently sequential like RNNs, they can still process speech data. This is usually done by transforming the speech into a suitable format, such as extracting features like Mel-frequency cepstral coefficients (MFCCs), pitch, tone, and amplitude.

* Feature Learning: ANNs learn to identify patterns and relationships in the speech data during the training process. The hidden layers of an ANN can capture complex relationships in the data, making them powerful for tasks like speech analysis where the input features may have intricate interdependencies.

* Classification or Regression Tasks: In the case of PD speech datasets, the typical task is to classify speech samples as indicative of either PD or a non-PD condition. ANNs achieve this through their output layer, which makes predictions based on the learned patterns in the data.

* Flexibility in Architecture: The architecture of an ANN can be varied (number of layers, number of neurons per layer) to suit the complexity of the task. For PD speech data, this flexibility allows for fine-tuning the network to better capture the nuances of speech affected by PD.

* Training and Optimization: ANNs are trained using backpropagation and gradient descent algorithms, where the model iteratively adjusts its weights to minimize the difference between the predicted output and the actual output. This training process is crucial for the network to learn the specific features of PD in speech data.

In summary, ANNs are utilized for PD speech data analysis by transforming the speech into a feature set that the network can process, learning complex relationships within this data, and ultimately classifying speech samples based on the presence or absence of Parkinson's Disease characteristics. The flexibility in designing the network and the power to learn intricate patterns make ANNs a valuable tool in speech analysis and other similar tasks.

### Creating different types of ANNs and evaluating their results

In [ ]:
def create_ann_model_1(input_shape):
    # Simple ANN model
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_ann_model_2(input_shape):
    # ANN model with more neurons
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_ann_model_3(input_shape):
    # ANN model with deeper layers
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to train and evaluate a model
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    return test_loss, test_accuracy

# Creating models
input_shape = (X_train.shape[1],)
model_1 = create_ann_model_1(input_shape)
model_2 = create_ann_model_2(input_shape)
model_3 = create_ann_model_3(input_shape)

# Training and evaluating models
loss_1, acc_1 = train_and_evaluate_model(model_1, X_train, y_train, X_test, y_test)
loss_2, acc_2 = train_and_evaluate_model(model_2, X_train, y_train, X_test, y_test)
loss_3, acc_3 = train_and_evaluate_model(model_3, X_train, y_train, X_test, y_test)

Epoch 1/10
12/12 [==============================] - 1s 23ms/step - loss: 0.7227 - accuracy: 0.6796 - val_loss: 0.4748 - val_accuracy: 0.7802
Epoch 2/10
12/12 [==============================] - 0s 8ms/step - loss: 0.5198 - accuracy: 0.7597 - val_loss: 0.4393 - val_accuracy: 0.7692
Epoch 3/10
12/12 [==============================] - 0s 11ms/step - loss: 0.5318 - accuracy: 0.7652 - val_loss: 0.4031 - val_accuracy: 0.7912
Epoch 4/10
12/12 [==============================] - 0s 9ms/step - loss: 0.5235 - accuracy: 0.7762 - val_loss: 0.4083 - val_accuracy: 0.8242
Epoch 5/10
12/12 [==============================] - 0s 7ms/step - loss: 0.4380 - accuracy: 0.7735 - val_loss: 0.3875 - val_accuracy: 0.8022
Epoch 6/10
12/12 [==============================] - 0s 7ms/step - loss: 0.4249 - accuracy: 0.8315 - val_loss: 0.3574 - val_accuracy: 0.8132
Epoch 7/10
12/12 [==============================] - 0s 8ms/step - loss: 0.3169 - accuracy: 0.8619 - val_loss: 0.3419 - val_accuracy: 0.8242
Epoch 8/10
12/12 [

In [ ]:
# Printing results
print("Model 1 - Loss:", loss_1, "Accuracy:", acc_1)
print("Model 2 - Loss:", loss_2, "Accuracy:", acc_2)
print("Model 3 - Loss:", loss_3, "Accuracy:", acc_3)

Model 1 - Loss: 0.33235326409339905 Accuracy: 0.8410596251487732
Model 2 - Loss: 0.3572838008403778 Accuracy: 0.8675496578216553
Model 3 - Loss: 0.3612525761127472 Accuracy: 0.8940397500991821


### **Definition of Recurrent Neural Networks (RNNs)**
Recurrent Neural Networks (RNNs) are a class of artificial neural networks designed to recognize patterns in sequences of data, such as text, genomes, handwriting, or time series data. Unlike standard feedforward neural networks, RNNs have a unique feature called loops, allowing information to persist. In an RNN, connections between nodes form a directed graph along a temporal sequence, enabling it to exhibit temporal dynamic behavior. This architecture makes them particularly well-suited for tasks where context and order in time are relevant.

### **RNNs in the Context of PD Speech Data**
When applying RNNs to PD (Parkinson's Disease) speech data, the focus is on capturing the temporal dynamics and contextual relationships within the speech. Here’s how RNNs function in this scenario:

* Temporal Feature Learning: RNNs are inherently suited for sequential data, making them ideal for speech analysis. They can process speech data as a sequence of time-based features, learning important temporal characteristics like changes in tone, pace, or intonation that might be indicative of PD.

* Capturing Dependencies Over Time: One of the key strengths of RNNs is their ability to connect previous information to the current task, which is crucial for speech data where the context and sequence of sounds carry significant information.

* Handling Variable-Length Input: Speech samples can vary in length, and RNNs can handle such variable-length input sequences effectively. This is particularly important for analyzing continuous speech data in a clinical setting where the speech duration may not be fixed.

* Sequence to Sequence Mapping: RNNs can map sequences to sequences, making them suitable for tasks like speech recognition or synthesis, and in the context of PD, for analyzing continuous speech patterns for signs of the disease.

* Challenges and Enhancements: Vanilla RNNs often suffer from problems like vanishing and exploding gradients. To mitigate these issues, advanced variants like LSTM (Long Short-Term Memory) and GRU (Gated Recurrent Units) are used. These models are better at capturing long-range dependencies and are more robust in training.

* Classification or Regression Tasks: In PD speech analysis, RNNs can be used for binary classification (PD or non-PD), or even for more nuanced tasks like staging the severity of PD based on speech characteristics.

* In summary, RNNs' ability to process sequential speech data, capture temporal dependencies, and handle variable-length input makes them particularly well-suited for analyzing PD speech data. By learning the intricate patterns and temporal sequences in speech affected by Parkinson's Disease, RNNs can play a crucial role in automated diagnostics and patient monitoring systems.

### Creating different types of RNNs and evaluating their results

In [ ]:
def create_rnn_model(input_shape):
    # Simple RNN model
    model = Sequential([
        SimpleRNN(64, return_sequences=True, input_shape=input_shape),
        SimpleRNN(64),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(input_shape):
    # LSTM model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        LSTM(64),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_gru_model(input_shape):
    # GRU model
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=input_shape),
        GRU(64),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to train and evaluate a model
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    return test_loss, test_accuracy

# Creating models
input_shape = (X_train.shape[1], X_train.shape[2])
rnn_model = create_rnn_model(input_shape)
lstm_model = create_lstm_model(input_shape)
gru_model = create_gru_model(input_shape)

# Training and evaluating models
rnn_loss, rnn_acc = train_and_evaluate_model(rnn_model, X_train, y_train, X_test, y_test)
lstm_loss, lstm_acc = train_and_evaluate_model(lstm_model, X_train, y_train, X_test, y_test)
gru_loss, gru_acc = train_and_evaluate_model(gru_model, X_train, y_train, X_test, y_test)

Epoch 1/10
12/12 [==============================] - 8s 442ms/step - loss: 0.5780 - accuracy: 0.6989 - val_loss: 0.4446 - val_accuracy: 0.7912
Epoch 2/10
12/12 [==============================] - 6s 451ms/step - loss: 0.4847 - accuracy: 0.7597 - val_loss: 0.4155 - val_accuracy: 0.7692
Epoch 3/10
12/12 [==============================] - 4s 361ms/step - loss: 0.4743 - accuracy: 0.7818 - val_loss: 0.4163 - val_accuracy: 0.7692
Epoch 4/10
12/12 [==============================] - 5s 440ms/step - loss: 0.4758 - accuracy: 0.7680 - val_loss: 0.4192 - val_accuracy: 0.8352
Epoch 5/10
12/12 [==============================] - 6s 441ms/step - loss: 0.4310 - accuracy: 0.7901 - val_loss: 0.3752 - val_accuracy: 0.8681
Epoch 6/10
12/12 [==============================] - 4s 329ms/step - loss: 0.4089 - accuracy: 0.8122 - val_loss: 0.3650 - val_accuracy: 0.8681
Epoch 7/10
12/12 [==============================] - 5s 426ms/step - loss: 0.3857 - accuracy: 0.8177 - val_loss: 0.3714 - val_accuracy: 0.8352
Epoch 

In [ ]:
# Printing results
print("RNN Model - Loss:", rnn_loss, "Accuracy:", rnn_acc)
print("LSTM Model - Loss:", lstm_loss, "Accuracy:", lstm_acc)
print("GRU Model - Loss:", gru_loss, "Accuracy:", gru_acc)

RNN Model - Loss: 0.4230723977088928 Accuracy: 0.8278145790100098
LSTM Model - Loss: 0.4217391908168793 Accuracy: 0.8344370722770691
GRU Model - Loss: 0.4541018009185791 Accuracy: 0.7947019934654236


### **Definition of Stacking**
Stacking, short for "stacked generalization," is an ensemble machine learning technique that combines multiple models to improve prediction accuracy. The key idea in stacking is to use a new model, known as the meta-model or blender, to learn how to best integrate the predictions of several base models.

In stacking, the initial level (or base level) consists of a variety of models which are trained on the full training dataset. These models can be diverse and include different types of machine learning algorithms. Each of these base models then makes predictions, but instead of using these predictions directly for the final output, they are used as input features for the next level.

### **Definition of Meta-Model**
The meta-model, which sits at the second level (or meta level), is trained on the outputs of the base models. The input to the meta-model is typically the predictions made by the base models on a holdout set (a portion of the training set not used to train the base models), and its output is the final prediction. The meta-model essentially learns the best way to combine the predictions from the base models to make a more accurate and robust prediction than any single base model could on its own.

### Doing Stacking of CNNs and Creating Meta Models(Logistic Regression, Random Forest, XGBoost, SVM and CNN)

#### Logistic Regression

In [ ]:
# Function to generate predictions for stacking
def generate_predictions(model, X):
    return model.predict(X).reshape(-1, 1)

# Training the CNN models
model_1 = create_cnn_model_1(input_shape)
model_2 = create_cnn_model_2(input_shape)
model_3 = create_cnn_model_3(input_shape)

# Training and generating predictions
model_1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

predictions_1 = generate_predictions(model_1, X_val)
predictions_2 = generate_predictions(model_2, X_val)
predictions_3 = generate_predictions(model_3, X_val)

# Stacking the predictions
stacked_predictions = np.hstack((predictions_1, predictions_2, predictions_3))

# Training meta-model on the stacked predictions
meta_model_lr = LogisticRegression()
meta_model_lr.fit(stacked_predictions, y_val)


# Evaluating the meta-model on the test set
# First, getting predictions from the base models on the test set
test_predictions_1 = generate_predictions(model_1, X_test)
test_predictions_2 = generate_predictions(model_2, X_test)
test_predictions_3 = generate_predictions(model_3, X_test)

# Stacking the test set predictions
stacked_test_predictions = np.hstack((test_predictions_1, test_predictions_2, test_predictions_3))

# Making final predictions with the meta-model
final_predictions_lr = meta_model_lr.predict(stacked_test_predictions)

# Evaluating accuracy
accuracy_lr = accuracy_score(y_test, final_predictions_lr)
print("Stacking with Logistic Regression - Accuracy:", accuracy_lr)

Epoch 1/25
6/6 [==============================] - 1s 72ms/step - loss: 0.7363 - accuracy: 0.7351
Epoch 2/25
6/6 [==============================] - 0s 67ms/step - loss: 0.5109 - accuracy: 0.8234
Epoch 3/25
6/6 [==============================] - 0s 66ms/step - loss: 0.4248 - accuracy: 0.7704
Epoch 4/25
6/6 [==============================] - 0s 64ms/step - loss: 0.3531 - accuracy: 0.8411
Epoch 5/25
6/6 [==============================] - 0s 64ms/step - loss: 0.3578 - accuracy: 0.8411
Epoch 6/25
6/6 [==============================] - 0s 83ms/step - loss: 0.2910 - accuracy: 0.8698
Epoch 7/25
6/6 [==============================] - 1s 95ms/step - loss: 0.2847 - accuracy: 0.8720
Epoch 8/25
6/6 [==============================] - 1s 95ms/step - loss: 0.2460 - accuracy: 0.8918
Epoch 9/25
6/6 [==============================] - 1s 85ms/step - loss: 0.2255 - accuracy: 0.9161
Epoch 10/25
6/6 [==============================] - 1s 88ms/step - loss: 0.2158 - accuracy: 0.9183
Epoch 11/25
6/6 [============

#### Random Forest

In [ ]:
# Function to generate predictions for stacking
def generate_predictions(model, X):
    return model.predict(X).reshape(-1, 1)

# Training the CNN models
model_1 = create_cnn_model_1(input_shape)
model_2 = create_cnn_model_2(input_shape)
model_3 = create_cnn_model_3(input_shape)

# Training and generating predictions
model_1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

predictions_1 = generate_predictions(model_1, X_val)
predictions_2 = generate_predictions(model_2, X_val)
predictions_3 = generate_predictions(model_3, X_val)

# Stacking the predictions
stacked_predictions = np.hstack((predictions_1, predictions_2, predictions_3))

# Training meta-model on the stacked predictions
meta_model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
meta_model_rf.fit(stacked_predictions, y_val)

# Evaluating the meta-model on the test set
# First, getting predictions from the base models on the test set
test_predictions_1 = generate_predictions(model_1, X_test)
test_predictions_2 = generate_predictions(model_2, X_test)
test_predictions_3 = generate_predictions(model_3, X_test)

# Stacking the test set predictions
stacked_test_predictions = np.hstack((test_predictions_1, test_predictions_2, test_predictions_3))

# Making final predictions with the meta-model
final_predictions_rf = meta_model_rf.predict(stacked_test_predictions)

# Evaluating accuracy
accuracy_rf = accuracy_score(y_test, final_predictions_rf)
print("Stacking with Random Forest - Accuracy:", accuracy_rf)

Epoch 1/25
6/6 [==============================] - 1s 61ms/step - loss: 0.7506 - accuracy: 0.6909
Epoch 2/25
6/6 [==============================] - 0s 62ms/step - loss: 0.4793 - accuracy: 0.8102
Epoch 3/25
6/6 [==============================] - 0s 63ms/step - loss: 0.3996 - accuracy: 0.8256
Epoch 4/25
6/6 [==============================] - 0s 58ms/step - loss: 0.3493 - accuracy: 0.8565
Epoch 5/25
6/6 [==============================] - 0s 62ms/step - loss: 0.3179 - accuracy: 0.8631
Epoch 6/25
6/6 [==============================] - 0s 62ms/step - loss: 0.2917 - accuracy: 0.8587
Epoch 7/25
6/6 [==============================] - 0s 62ms/step - loss: 0.2524 - accuracy: 0.8940
Epoch 8/25
6/6 [==============================] - 0s 62ms/step - loss: 0.2470 - accuracy: 0.8918
Epoch 9/25
6/6 [==============================] - 0s 54ms/step - loss: 0.2168 - accuracy: 0.9095
Epoch 10/25
6/6 [==============================] - 0s 53ms/step - loss: 0.2022 - accuracy: 0.9205
Epoch 11/25
6/6 [============

#### XGBoost

In [ ]:
# Function to generate predictions for stacking
def generate_predictions(model, X):
    return model.predict(X).reshape(-1, 1)

# Training the CNN models
model_1 = create_cnn_model_1(input_shape)
model_2 = create_cnn_model_2(input_shape)
model_3 = create_cnn_model_3(input_shape)

# Training and generating predictions
model_1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

predictions_1 = generate_predictions(model_1, X_val)
predictions_2 = generate_predictions(model_2, X_val)
predictions_3 = generate_predictions(model_3, X_val)

# Stacking the predictions
stacked_predictions = np.hstack((predictions_1, predictions_2, predictions_3))

# Training different meta-models on the stacked predictions
meta_model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
meta_model_xgb.fit(stacked_predictions, y_val)

# Evaluating the meta-model on the test set
# First, getting predictions from the base models on the test set
test_predictions_1 = generate_predictions(model_1, X_test)
test_predictions_2 = generate_predictions(model_2, X_test)
test_predictions_3 = generate_predictions(model_3, X_test)

# Stacking the test set predictions
stacked_test_predictions = np.hstack((test_predictions_1, test_predictions_2, test_predictions_3))

# Making final predictions with the meta-model
final_predictions_xgb = meta_model_xgb.predict(stacked_test_predictions)

# Evaluating accuracy
accuracy_xgb = accuracy_score(y_test, final_predictions_xgb)
print("Stacking with XGBoost - Accuracy:", accuracy_xgb)

Epoch 1/25
6/6 [==============================] - 1s 60ms/step - loss: 0.7014 - accuracy: 0.7174
Epoch 2/25
6/6 [==============================] - 0s 55ms/step - loss: 0.4863 - accuracy: 0.7837
Epoch 3/25
6/6 [==============================] - 0s 57ms/step - loss: 0.3960 - accuracy: 0.8344
Epoch 4/25
6/6 [==============================] - 0s 53ms/step - loss: 0.3494 - accuracy: 0.8499
Epoch 5/25
6/6 [==============================] - 0s 53ms/step - loss: 0.3248 - accuracy: 0.8521
Epoch 6/25
6/6 [==============================] - 0s 57ms/step - loss: 0.3005 - accuracy: 0.8653
Epoch 7/25
6/6 [==============================] - 0s 53ms/step - loss: 0.2887 - accuracy: 0.8808
Epoch 8/25
6/6 [==============================] - 0s 65ms/step - loss: 0.2597 - accuracy: 0.8808
Epoch 9/25
6/6 [==============================] - 0s 69ms/step - loss: 0.2312 - accuracy: 0.9051
Epoch 10/25
6/6 [==============================] - 1s 96ms/step - loss: 0.2026 - accuracy: 0.9272
Epoch 11/25
6/6 [============

#### SVM

In [ ]:
# Function to generate predictions for stacking
def generate_predictions(model, X):
    return model.predict(X).reshape(-1, 1)

# Training the CNN models
model_1 = create_cnn_model_1(input_shape)
model_2 = create_cnn_model_2(input_shape)
model_3 = create_cnn_model_3(input_shape)

# Training and generating predictions
model_1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

predictions_1 = generate_predictions(model_1, X_val)
predictions_2 = generate_predictions(model_2, X_val)
predictions_3 = generate_predictions(model_3, X_val)

# Stacking the predictions
stacked_predictions = np.hstack((predictions_1, predictions_2, predictions_3))

# Training different meta-models on the stacked predictions

meta_model_svm = SVC(probability=True)
meta_model_svm.fit(stacked_predictions, y_val)

# Evaluate the meta-model on the test set
# First, getting predictions from the base models on the test set
test_predictions_1 = generate_predictions(model_1, X_test)
test_predictions_2 = generate_predictions(model_2, X_test)
test_predictions_3 = generate_predictions(model_3, X_test)

# Stacking the test set predictions
stacked_test_predictions = np.hstack((test_predictions_1, test_predictions_2, test_predictions_3))

# Making final predictions with the meta-model
final_predictions_svm = meta_model_svm.predict(stacked_test_predictions)

# Evaluating accuracy
accuracy_svm = accuracy_score(y_test, final_predictions_svm)
print("Stacking with SVM - Accuracy:", accuracy_svm)

Epoch 1/25
6/6 [==============================] - 2s 70ms/step - loss: 0.6582 - accuracy: 0.7329
Epoch 2/25
6/6 [==============================] - 0s 62ms/step - loss: 0.4937 - accuracy: 0.8079
Epoch 3/25
6/6 [==============================] - 0s 64ms/step - loss: 0.4079 - accuracy: 0.8278
Epoch 4/25
6/6 [==============================] - 0s 64ms/step - loss: 0.3474 - accuracy: 0.8521
Epoch 5/25
6/6 [==============================] - 0s 61ms/step - loss: 0.3181 - accuracy: 0.8587
Epoch 6/25
6/6 [==============================] - 0s 62ms/step - loss: 0.3021 - accuracy: 0.8565
Epoch 7/25
6/6 [==============================] - 0s 65ms/step - loss: 0.2540 - accuracy: 0.8985
Epoch 8/25
6/6 [==============================] - 1s 89ms/step - loss: 0.2449 - accuracy: 0.9007
Epoch 9/25
6/6 [==============================] - 1s 102ms/step - loss: 0.2290 - accuracy: 0.9007
Epoch 10/25
6/6 [==============================] - 1s 101ms/step - loss: 0.2001 - accuracy: 0.9183
Epoch 11/25
6/6 [==========

#### CNN (As meta model)

In [ ]:
# Function to generate predictions for stacking
def generate_predictions(model, X):
    return model.predict(X).reshape(-1, 1)

# Training the CNN models
model_1 = create_cnn_model_1(input_shape)
model_2 = create_cnn_model_2(input_shape)
model_3 = create_cnn_model_3(input_shape)

# Training and generating predictions
model_1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model_3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

predictions_1 = generate_predictions(model_1, X_val)
predictions_2 = generate_predictions(model_2, X_val)
predictions_3 = generate_predictions(model_3, X_val)

test_predictions_1 = generate_predictions(model_1, X_test)
test_predictions_2 = generate_predictions(model_2, X_test)
test_predictions_3 = generate_predictions(model_3, X_test)

# Stacking the test set predictions
stacked_test_predictions = np.hstack((test_predictions_1, test_predictions_2, test_predictions_3))

# Stacking the predictions
stacked_predictions = np.hstack((predictions_1, predictions_2, predictions_3))

# Reshaping stacked predictions for CNN input
stacked_predictions_cnn = np.expand_dims(stacked_predictions, axis=2)  # Add an extra dimension

# Similarly, reshaping the test set predictions
stacked_test_predictions_cnn = np.expand_dims(stacked_test_predictions, axis=2)

def create_cnn_meta_model(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Creating the CNN meta-model
input_shape_meta = (stacked_predictions_cnn.shape[1], stacked_predictions_cnn.shape[2])
meta_model_cnn = create_cnn_meta_model(input_shape_meta)

# Training the CNN meta-model
meta_model_cnn.fit(stacked_predictions_cnn, y_val, epochs=10, batch_size=32)

# Making final predictions with the meta-model
final_predictions_cnn = meta_model_cnn.predict(stacked_test_predictions_cnn)

# Threshold the predictions
final_predictions_cnn = (final_predictions_cnn > 0.5).astype(int).flatten()

# Evaluating accuracy
accuracy_cnn = accuracy_score(y_test, final_predictions_cnn)
print("Stacking with CNN Meta-Model - Accuracy:", accuracy_cnn)

Epoch 1/25
6/6 [==============================] - 2s 97ms/step - loss: 0.6441 - accuracy: 0.7020
Epoch 2/25
6/6 [==============================] - 1s 98ms/step - loss: 0.4819 - accuracy: 0.7903
Epoch 3/25
6/6 [==============================] - 1s 99ms/step - loss: 0.3917 - accuracy: 0.8146
Epoch 4/25
6/6 [==============================] - 1s 90ms/step - loss: 0.3546 - accuracy: 0.8631
Epoch 5/25
6/6 [==============================] - 1s 86ms/step - loss: 0.3233 - accuracy: 0.8521
Epoch 6/25
6/6 [==============================] - 1s 98ms/step - loss: 0.2932 - accuracy: 0.8653
Epoch 7/25
6/6 [==============================] - 1s 96ms/step - loss: 0.2674 - accuracy: 0.8764
Epoch 8/25
6/6 [==============================] - 0s 67ms/step - loss: 0.2442 - accuracy: 0.8808
Epoch 9/25
6/6 [==============================] - 0s 64ms/step - loss: 0.2226 - accuracy: 0.9095
Epoch 10/25
6/6 [==============================] - 0s 62ms/step - loss: 0.1975 - accuracy: 0.9272
Epoch 11/25
6/6 [============